# Intelligent Engagement Classification in CRM Activities

This notebook implements an **AI-driven pipeline** to classify CRM activities as **Engaged**, **Not Engaged**, or **Non-Determinable (ND)**.

The goal is to go beyond raw CRM activity logs and determine whether a **real human-to-human interaction** actually occurred, using a combination of:
- Data cleaning & heuristics
- Rule-based pre-labeling
- Large Language Models (LLMs) with agentic reasoning
- Cost-efficient bulk inference

The final output is a new column, `trueai_engaged`, designed for **sales performance analytics and reporting**.


## Dataset Upload and Loading

The dataset contains CRM activities synced from a demo sales organization.  
Each row represents a logged activity such as an email, call, task, or meeting.

We begin by uploading and loading the CSV file into a Pandas DataFrame.


In [ ]:
import pandas as pd

# Upload file in Colab
from google.colab import files
uploaded = files.upload()

# Load dataset
df = pd.read_csv("verifictus_activities.csv")




Saving verifictus_activities.csv to verifictus_activities.csv


## Dataset Structure and Scope

We restrict the dataset to the three core fields required for engagement analysis:

- `crm_subject`: Activity title (often includes automation tags such as `[Outreach]`)
- `crm_description`: Raw unstructured text describing the interaction
- `crm_subtype`: Activity category (Email, Call, Task, Event)

This reduction removes irrelevant CRM metadata and focuses the analysis on signal-bearing fields.


In [ ]:
df = df.iloc[:, :3]
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
df.head()

Rows: 5000
Columns: 3


,crm_subject,crm_description,crm_subtype
0,[Outreach] [Email] [Out] 12 Re: Cut Onboarding...,"From: ""Anthony D'Angelo"" <adangelo@verifictus....",Email
1,1 - Medtronic - CA - **,NaN,Task
2,[Outreach] [Email] [Out] 1 GTM strategy,"From: ""Steve Brady"" <sbrady@verifictus.com>\nT...",Email
3,4 - Medtronic - NH,NaN,Task
4,convert,NaN,Task


## Missing Data Inspection

CRM systems frequently contain incomplete or system-generated records.

This step quantifies:
- Missing values per column
- Percentage of missing data

This analysis helps justify the introduction of a **Non-Determinable (ND)** class later in the pipeline.


In [ ]:
missing_summary = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_percent": (df.isna().mean() * 100).round(2)
})

missing_summary



,missing_count,missing_percent
crm_subject,0,0.00
crm_description,1238,24.76
crm_subtype,0,0.00


## CRM Activity Type Distribution

We analyze the distribution of activity subtypes (`Email`, `Call`, `Task`, `Event`) to:

- Understand data imbalance
- Anticipate classification difficulty per subtype
- Validate assumptions about sales engagement volume

Salesforce system artifacts are removed to avoid skewing the analysis.


In [ ]:
subtype_counts = df["crm_subtype"].value_counts(dropna=False)

subtype_percentages = (subtype_counts / subtype_counts.sum() * 100).round(2)

subtype_summary = pd.DataFrame({
    'count': subtype_counts,
    'percentage': subtype_percentages
})

subtype_summary


,count,percentage
crm_subtype,,
Task,2748,54.96
Email,1149,22.98
Call,881,17.62
Event,221,4.42
Salesforce,1,0.02


In [ ]:
subtype_summary = subtype_summary[subtype_summary.index != ' Salesforce']
subtype_summary

,count,percentage
crm_subtype,,
Task,2748,54.96
Email,1149,22.98
Call,881,17.62
Event,221,4.42


## CRM Description Length Analysis

The length of `crm_description` is a strong proxy for information richness.

We compute:
- Character length per description
- Summary statistics
- Longest descriptions

This step helps identify:
- Boilerplate system messages
- Rich conversational logs suitable for LLM reasoning


In [ ]:
df["description_length"] = df["crm_description"].fillna("").str.len()

df["description_length"].describe()
df.sort_values("description_length", ascending=False).head(10)[
    ["crm_subtype", "crm_subject", "description_length"]
]



,crm_subtype,crm_subject,description_length
129,Task,Re: CA & verifictus Continuation,2135
2819,Task,[People.ai | Outbound] Re: A verifictus Welcome!,2132
1459,Task,Re: PDF of contract,2129
2455,Task,Re: 1:1 Andrew | Jack (verifictus) - Brief notes,2129
980,Task,Re: Time to speak,2126
2293,Task,Re: CPB - Shift 2023,2123
3320,Task,[People.ai | Outbound] Re: Kick off call,2120
234,Task,"Re: Draft contract from verifictus Software, I...",2120
2664,Task,RE: verifictus Licenses,2117
2736,Task,Re: verifictus x Builder.ai - Completed RFP,2114


## Duplicate Detection Using Row-Level Hashing

High-volume CRM systems often contain duplicated activity records.

To ensure data integrity:
- Each row is hashed using its full content
- Duplicates are removed deterministically

This guarantees that each activity is classified exactly once.


In [ ]:
import hashlib

def generate_row_hash(row):
    # Convert all row values to string and concatenate them.
    # Using a separator ('|') helps prevent collisions where different combinations
    # of column values might result in the same concatenated string without a separator.
    # It's important to handle NaN values gracefully, e.g., by converting them to empty strings.
    row_string = '|'.join(row.fillna('').astype(str))
    return hashlib.md5(row_string.encode('utf-8')).hexdigest()

# Apply the function to each row to create the 'unique_id' column
df['unique_id'] = df.apply(generate_row_hash, axis=1)

# Get the number of rows before removing duplicates
num_rows_before = df.shape[0]

# Remove duplicate rows based on the 'unique_id' column
df = df.drop_duplicates(subset=['unique_id'])

# Get the number of rows after removing duplicates
num_rows_after = df.shape[0]

print(f"Number of rows before adding unique_id and removing duplicates: {num_rows_before}")
print(f"Number of rows after removing duplicates: {num_rows_after}")
print("DataFrame head after removing duplicates:")
print(df.head())

Number of rows before adding unique_id and removing duplicates: 5000
Number of rows after removing duplicates: 3810
DataFrame head after removing duplicates:
                                         crm_subject  \
0  [Outreach] [Email] [Out] 12 Re: Cut Onboarding...   
1                            1 - Medtronic - CA - **   
2            [Outreach] [Email] [Out] 1 GTM strategy   
3                                 4 - Medtronic - NH   
4                                            convert   

                                     crm_description crm_subtype  \
0  From: "Anthony D'Angelo" <adangelo@verifictus....       Email   
1                                                NaN        Task   
2  From: "Steve Brady" <sbrady@verifictus.com>\nT...       Email   
3                                                NaN        Task   
4                                                NaN        Task   

   description_length                         unique_id  
0                2073  9e7335a956dc791

In [ ]:
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
df.head()

Rows: 3810
Columns: 5


,crm_subject,crm_description,crm_subtype,description_length,unique_id
0,[Outreach] [Email] [Out] 12 Re: Cut Onboarding...,"From: ""Anthony D'Angelo"" <adangelo@verifictus....",Email,2073,9e7335a956dc791472795a64f582c86c
1,1 - Medtronic - CA - **,NaN,Task,0,8ee94e799271308bb6c900c3967fd211
2,[Outreach] [Email] [Out] 1 GTM strategy,"From: ""Steve Brady"" <sbrady@verifictus.com>\nT...",Email,1652,77c787c0586c7946ec1c106f437c54bc
3,4 - Medtronic - NH,NaN,Task,0,668bd7974c64eff06653d21c5d8d5417
4,convert,NaN,Task,0,147cb6d8ebc52d758dd5f5e63052a9e9


In [ ]:
df = df.drop(columns=['unique_id'])
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
df.head()

Rows: 3810
Columns: 4


,crm_subject,crm_description,crm_subtype,description_length
0,[Outreach] [Email] [Out] 12 Re: Cut Onboarding...,"From: ""Anthony D'Angelo"" <adangelo@verifictus....",Email,2073
1,1 - Medtronic - CA - **,NaN,Task,0
2,[Outreach] [Email] [Out] 1 GTM strategy,"From: ""Steve Brady"" <sbrady@verifictus.com>\nT...",Email,1652
3,4 - Medtronic - NH,NaN,Task,0
4,convert,NaN,Task,0


## CRM Description Cleaning

Raw CRM text is noisy and contains:
- Unsubscribe blocks
- Social media footers
- HTML tags and entities
- URLs and email addresses

We apply aggressive cleaning to:
- Preserve semantic meaning
- Remove non-informative boilerplate
- Improve LLM signal quality


In [ ]:
import re

def clean_description(text):
    if pd.isna(text) or text == '':
        return ''

    # 1. Remove unsubscribe blocks (case-insensitive)
    text = re.sub(r'((unsubscribe|preference center|opt-out|manage your preferences|don\'t want to receive|no longer wish to receive).{0,100}?(click here|here|below|link|at|url|this link|to opt out))', '', text, flags=re.IGNORECASE | re.DOTALL)
    text = re.sub(r'(Sent by:.+?(Unsubscribe|opt-out))', '', text, flags=re.IGNORECASE | re.DOTALL)
    text = re.sub(r'(\b(unsubscribe|opt out)\b)', '', text, flags=re.IGNORECASE)

    # 2. Remove social media blocks (common social links/icons/phrases)
    text = re.sub(r'((follow us|connect with us|find us on|social media links?|share on|check us out on).{0,100}?(facebook|twitter|linkedin|instagram|youtube|blog|website|pinterest|snapchat|tiktok|telegram))', '', text, flags=re.IGNORECASE | re.DOTALL)

    # 3. Remove URLs and email addresses
    text = re.sub(r'\b(?:https?://|www\.)\S+\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text, flags=re.IGNORECASE)

    # 4. Remove HTML tags (e.g., <p>, <div>, <a>, <br/>)
    text = re.sub(r'<[^>]+>', '', text)

    # 5. Remove HTML entities (e.g., &nbsp;, &amp;, &lt;, &gt;)
    text = re.sub(r'&[a-z0-9]+;', '', text, flags=re.IGNORECASE)

    # 6. Remove any remaining inline CSS styles if not caught by HTML tag removal
    # This is a more specific regex for style attributes, usually covered by <[^>]+>
    text = re.sub(r'\s*style=\"([^"]*)\"', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s*style=\'([^\']*)\'', '', text, flags=re.IGNORECASE)

    # Remove any excessive whitespace created by removals
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply the cleaning function to the 'crm_description' column
df['crm_description_cleaned'] = df['crm_description'].apply(clean_description)

# Calculate the new description length after cleaning
df['description_length_cleaned'] = df['crm_description_cleaned'].str.len()

# Display some examples to verify the cleaning
print("Original vs. Cleaned crm_description examples after HTML/CSS removal:")
print(df[['crm_description', 'crm_description_cleaned', 'description_length', 'description_length_cleaned']].head())

Original vs. Cleaned crm_description examples after HTML/CSS removal:
                                     crm_description  \
0  From: "Anthony D'Angelo" <adangelo@verifictus....   
1                                                NaN   
2  From: "Steve Brady" <sbrady@verifictus.com>\nT...   
3                                                NaN   
4                                                NaN   

                             crm_description_cleaned  description_length  \
0  From: "Anthony D'Angelo" <> To: "Clinton Tiggs...                2073   
1                                                                      0   
2  From: "Steve Brady" <> To: "Melanie Marcus" <>...                1652   
3                                                                      0   
4                                                                      0   

   description_length_cleaned  
0                        1531  
1                           0  
2                        1097  
3       

## Cleaning Validation

We compare:
- Original vs cleaned descriptions
- Text length before and after cleaning

This sanity check ensures that:
- Meaningful content is preserved
- Only noise is removed


In [ ]:
print(df.loc[0, 'crm_description_cleaned'])

From: "Anthony D'Angelo" <> To: "Clinton Tiggs" <> Hi Clinton, Unfortunately, we havent been able to connect yet. I totally understand; life gets busy. I hope you dont take offense to my persistence. That said, I wouldnt reach out if I didnt believe Lessonly could help your team.As a quick recap, these are the common challenges I continue to hear within call centers:-Onboarding new agents takes too long-CSAT scores are not where they should be-Lack of microburst content for in the moment training Think you can spare a few minutes to connect me with the person responsible for training and development? Thanks,Anthony verifictus Anthony DAngelo ( ) | 631.356.9542 ( ) | LinkedIn ( / )Twitter ( )Instagram ( / ) If you'd like me to stop sending you emails, please click here ( %% ) On Wed, Dec 14, 2022 at 11:53 am, Anthony D'Angelo wrote: Hi Clinton, Hope youre having a great Wednesday! Just seeing if you got my previous note. Are you open to a quick call? Otherwise, is there someone on your 

In [ ]:
df.sort_values("description_length_cleaned", ascending=False).head(10)[
    ["crm_description_cleaned", "description_length_cleaned"]
]

,crm_description_cleaned,description_length_cleaned
3233,To: Subject: verifictus Learning | Experience ...,2068
168,To: Subject: November: verifictus Capabilities...,2050
3199,Hi Michael Thank you once again for hosting To...,2044
3321,To: Subject: Your verifictus Shift Recap Messa...,2041
3524,"5 mins this week isn't impossible, is it? ----...",2035
1680,To: Subject: June: verifictus Capabilities Upd...,2033
4070,To: Subject: Introducing verifictus Knowledge!...,2025
781,From: Subject: Accepted: Equals & verifictus |...,2022
1212,"RECORDING: ATTENDEES: Jack Leahy, Account Mana...",2017
4374,"MIKE WELLS Business Change, Product Owner (CRM...",2015


In [ ]:
print(df.loc[3233, 'crm_description_cleaned'])

To: Subject: verifictus Learning | Experience Updates Message: Hi [[Recipient.Name]] I hope this message finds you well. By now you have heard about the exciting upgrades available for verifictus Learning customers. With our Fall Release we announced the rollout of an enhanced authentication experience, adding a single sign-on capability that will streamline the login process across your verifictus Learning environment and our customer-facing portals. This enhancement unlocks our ability to centralize all customers on single learning, community, documentation, and chat portals, providing a more consistent and actively managed experience for all of our customers. As part of this next evolution, there are key changes we want to share with you. Enhanced Authentication upgrade unlocks access to verifictus University & verifictus Community Effective date: November 15th Enhanced Authentication is currently available to all verifictus Learning customers. This release will unlock access to por

In [ ]:
df.head()

,crm_subject,crm_description,crm_subtype,description_length,crm_description_cleaned,description_length_cleaned
0,[Outreach] [Email] [Out] 12 Re: Cut Onboarding...,"From: ""Anthony D'Angelo"" <adangelo@verifictus....",Email,2073,"From: ""Anthony D'Angelo"" <> To: ""Clinton Tiggs...",1531
1,1 - Medtronic - CA - **,NaN,Task,0,,0
2,[Outreach] [Email] [Out] 1 GTM strategy,"From: ""Steve Brady"" <sbrady@verifictus.com>\nT...",Email,1652,"From: ""Steve Brady"" <> To: ""Melanie Marcus"" <>...",1097
3,4 - Medtronic - NH,NaN,Task,0,,0
4,convert,NaN,Task,0,,0


## Phase 2: Modeling Dataset Preparation

We create a clean modeling dataset containing:
- `crm_subject`
- `crm_subtype`
- Cleaned `crm_description`

This dataset serves as the input for both:
- Rule-based pre-labeling
- LLM-based agentic classification


In [ ]:
df2 = df[['crm_subject', 'crm_subtype', 'crm_description_cleaned']].copy()
df2 = df2.rename(columns={'crm_description_cleaned': 'crm_description'})

print("Rows:", df2.shape[0])
print("Columns:", df2.shape[1])
print("DataFrame head of the new copy:")
df2.head()

Rows: 3810
Columns: 3
DataFrame head of the new copy:


,crm_subject,crm_subtype,crm_description
0,[Outreach] [Email] [Out] 12 Re: Cut Onboarding...,Email,"From: ""Anthony D'Angelo"" <> To: ""Clinton Tiggs..."
1,1 - Medtronic - CA - **,Task,
2,[Outreach] [Email] [Out] 1 GTM strategy,Email,"From: ""Steve Brady"" <> To: ""Melanie Marcus"" <>..."
3,4 - Medtronic - NH,Task,
4,convert,Task,


## Rule-Based Pre-Labeling (Heuristic Layer)

Before invoking an LLM, we apply **lightweight heuristic rules** to:

- Capture obvious engagement signals
- Filter out clear non-engagement cases
- Reduce ambiguity and LLM workload

This step improves:
- Cost efficiency
- Overall pipeline robustness

## Classification Logic Summary

**ENGAGED**
- Replied emails (`Re:`)
- Calls with conversational cues
- Meetings that actually occurred

**NOT ENGAGED**
- Outreach sequences
- Unanswered calls / voicemails
- Internal sales tasks

**ND**
- Empty, short, or system-generated logs


In [ ]:
import numpy as np

def classify_interaction(row):
    subject = str(row['crm_subject']).lower() if pd.notna(row['crm_subject']) else ''
    description = str(row['crm_description']).lower() if pd.notna(row['crm_description']) else ''
    subtype = str(row['crm_subtype']).lower() if pd.notna(row['crm_subtype']) else ''

    # Rule 3: ND (Non-Determinable)
    # Check original crm_description as 'crm_description_cleaned' might be empty after cleaning
    original_description = str(df.loc[row.name, 'crm_description']).lower() if pd.notna(df.loc[row.name, 'crm_description']) else ''

    if not original_description or original_description == 'convert' or original_description == 'call':
        return 'ND'

    # Rule 1: ENGAGED
    # Emails
    if subtype == 'email' and ('re:' in subject or '?' in description):
        return 'ENGAGED'

    # Calls
    if subtype == 'call' and ('spoke with prospect' in description or 'they mentioned budget' in description):
        return 'ENGAGED'

    # Events
    if subtype == 'event' and ('meeting took place' in description or 'event attended' in description):
        # Assuming 'event attended' or similar phrases indicate actual meeting
        return 'ENGAGED'

    # Rule 2: NOT ENGAGED
    # Emails
    if subtype == 'email' and (('outreach]' in subject and '[out]' in subject) or '[[name]]' in description or 'unreplied' in description):
        return 'NOT ENGAGED'

    # Calls
    if subtype == 'call' and ('left voicemail' in description or 'no answer' in description or 'busy' in description or 'wrong number' in description):
        return 'NOT ENGAGED'

    # Tasks
    if subtype == 'task' and ('research lead' in description or 'follow up next week' in description):
        return 'NOT ENGAGED'

    # If none of the above specific rules apply, but there is some description, classify as NOT ENGAGED for general cases
    if description and description != 'nan':
        return 'NOT ENGAGED'

    return 'ND'


# Apply the classification function to the DataFrame
df2['classification'] = df2.apply(classify_interaction, axis=1)

# Display the count of each classification
print("Classification Distribution:")
print(df2['classification'].value_counts())

# Display a sample of the DataFrame with the new classification column
print("\nDataFrame head with classification:")
print(df2[['crm_subject', 'crm_subtype', 'crm_description', 'classification']].head())

Classification Distribution:
classification
NOT ENGAGED    2761
ENGAGED         956
ND               93
Name: count, dtype: int64

DataFrame head with classification:
                                         crm_subject crm_subtype  \
0  [Outreach] [Email] [Out] 12 Re: Cut Onboarding...       Email   
1                            1 - Medtronic - CA - **        Task   
2            [Outreach] [Email] [Out] 1 GTM strategy       Email   
3                                 4 - Medtronic - NH        Task   
4                                            convert        Task   

                                     crm_description classification  
0  From: "Anthony D'Angelo" <> To: "Clinton Tiggs...        ENGAGED  
1                                                                ND  
2  From: "Steve Brady" <> To: "Melanie Marcus" <>...        ENGAGED  
3                                                                ND  
4                                                                ND  


## Activity Identifier (hs_object_id)

Each activity is assigned a unique `hs_object_id` derived from its row index.

This identifier:
- Ensures prediction isolation
- Enables safe merging of LLM outputs
- Mimics production CRM object IDs


In [ ]:
df2['hs_object_id'] = df2.index
print("Added 'hs_object_id' column to df2 using its index.")

Added 'hs_object_id' column to df2 using its index.


In [ ]:
df2.head()

,crm_subject,crm_subtype,crm_description,classification,hs_object_id
0,[Outreach] [Email] [Out] 12 Re: Cut Onboarding...,Email,"From: ""Anthony D'Angelo"" <> To: ""Clinton Tiggs...",ENGAGED,0
1,1 - Medtronic - CA - **,Task,,ND,1
2,[Outreach] [Email] [Out] 1 GTM strategy,Email,"From: ""Steve Brady"" <> To: ""Melanie Marcus"" <>...",ENGAGED,2
3,4 - Medtronic - NH,Task,,ND,3
4,convert,Task,,ND,4


## LLM Configuration (Google Gemini)

We use **Gemini Flash** for fast, cost-effective inference.

Key design choices:
- JSON-only structured output
- Deterministic mapping using `hs_object_id`
- Optimized for batch processing


In [ ]:
pip install -q -U google-generativeai

In [ ]:
import os
import google.generativeai as genai

# Avoid proxy/network issues in Colab
os.environ['NO_PROXY'] = 'googleapis.com'

# Configure Gemini API
genai.configure(api_key="API_KEY")

# Initialize Gemini Flash model with JSON-only output
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    generation_config={
        "response_mime_type": "application/json"
    }
)


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


## Agentic Reasoning Prompt

The LLM is instructed to act as a **Sales Operations Auditor**, not a classifier.

This agent:
- Analyzes linguistic evidence of human interaction
- Distinguishes signal from CRM noise
- Applies consistent reasoning rules across records


In [ ]:
import json

def build_prompt(records):
    prompt = f"""
ROLE: You are an expert Sales Operations Auditor.

TASK:
Analyze each CRM activity and classify it as:
- ENGAGED
- NOT ENGAGED
- ND (Non-Determinable)

AGENTIC REASONING RULES:

ENGAGED:
- Emails with replies ("Re:")
- Calls showing 2-way interaction
- Meetings that clearly took place

NOT ENGAGED:
- Outreach sequences
- Unanswered calls / voicemails
- Automated or internal tasks

ND:
- Empty, too short, or ambiguous descriptions

IMPORTANT:
- Each record must be evaluated independently.
- Do NOT let context from one record influence another.

DATA:
{json.dumps(records)}

RETURN FORMAT:
JSON array of objects with:
- hs_object_id
- trueai_engaged
"""
    return prompt


## Bulk Inference Strategy

To minimize API cost and latency:
- Activities are processed in batches
- Each batch is sent as a single prompt
- Predictions remain isolated per record

Rate limiting ensures compliance with API quotas.


In [ ]:
import time
import pandas as pd

def classify_batch(batch_df):
    records = batch_df[
        ['hs_object_id', 'crm_subject', 'crm_description', 'crm_subtype']
    ].to_dict(orient="records")

    prompt = build_prompt(records)

    try:
        response = model.generate_content(
            prompt,
            request_options={"timeout": 30}
        )
        return json.loads(response.text)

    except Exception as e:
        print(f"Batch error: {e}")
        return [
            {"hs_object_id": r["hs_object_id"], "trueai_engaged": "ERROR"}
            for r in records
        ]


In [ ]:
def run_full_process(df, batch_size=20):
    all_results = []

    if "hs_object_id" not in df.columns:
        df["hs_object_id"] = range(len(df))

    for i in range(0, len(df), batch_size):
        print(f"Processing rows {i} to {i + batch_size}")
        batch = df.iloc[i:i + batch_size]

        batch_results = classify_batch(batch)
        all_results.extend(batch_results)

        # Rate limiting (Free Tier safe)
        time.sleep(6)

    return pd.DataFrame(all_results)


## Structured Output Integration

The model returns a JSON array:
- One entry per `hs_object_id`
- Each mapped to `trueai_engaged`

Results are merged back into the dataset with full traceability.


In [ ]:
# Run inference on a subset (example)
df_test = df2.head(100).copy()

results_df = run_full_process(df_test)

# Ensure correct type for merging
results_df["hs_object_id"] = results_df["hs_object_id"].astype(int)

# Merge predictions back to original dataframe
df_final = df_test.merge(
    results_df,
    on="hs_object_id",
    how="left"
)


Processing rows 0 to 20
Processing rows 20 to 40
Processing rows 40 to 60
Processing rows 60 to 80
Processing rows 80 to 100


## Final Output

The final dataset includes a new column:

`trueai_engaged ∈ {ENGAGED, NOT ENGAGED, ND}`

This column serves as:
- A reliable engagement signal
- A filter for sales success reporting
- A foundation for downstream analytics and dashboards


In [ ]:
print(df_final["trueai_engaged"].value_counts())

trueai_engaged
NOT ENGAGED    77
ENGAGED        17
ND              6
Name: count, dtype: int64


In [ ]:
print(
    df_final[["hs_object_id", "trueai_engaged"]]
    .to_json(orient="records", indent=4)
)


[
    {
        "hs_object_id":0,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":1,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":2,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":3,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":4,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":5,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":6,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":7,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":8,
        "trueai_engaged":"ENGAGED"
    },
    {
        "hs_object_id":9,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":10,
        "trueai_engaged":"NOT ENGAGED"
    },
    {
        "hs_object_id":11,
        "trueai_engaged":"ENGAGED"
    },
    {
        "hs_object_id":12,
        "trueai_engaged":"NOT ENGAG

# Model Evaluation
This section evaluates the performance of our LLM-based engagement classifier.
We compare its predictions against a **human-labeled ground truth dataset** to measure accuracy, precision, recall, and F1-score.  
Error analysis is also performed to identify patterns in misclassifications.


## 1. Create Human-Labeled Evaluation Set

Since the LLM predictions need to be evaluated, we first create a **human-labeled sample**.
We randomly select a subset of CRM activities (~200 records) and label them manually as:
- ENGAGED → confirmed human-to-human interaction
- NOT ENGAGED → unidirectional outreach or noise
- ND → ambiguous or insufficient information

This subset serves as our **gold standard** for evaluation.


In [ ]:
# Random sample for human annotation
df_eval = df2.sample(n=200, random_state=42).copy()

# Keep only relevant columns
df_eval = df_eval[
    ['hs_object_id', 'crm_subject', 'crm_subtype', 'crm_description']
]

df_eval.head()


,hs_object_id,crm_subject,crm_subtype,crm_description
1375,1375,Digital Intent - Renewal - 2023-06-14,Task,Customer never fully implemented.
4179,4179,CGI: revoire les scenarios verifictus pour le ...,Event,Merci Abdelaziz pour notre echange. On prevoit...
4850,4850,[Outreach] [Email] [Out] 6 Re: Katherine x ver...,Email,"From: ""Sally Abel"" <> To: ""Katherine Foy"" <> H..."
1203,1203,[Outreach] [Email] [In] Error Occurred During ...,Email,"From: FlowApplication To: """" <> Debug the fail..."
4253,4253,[Outreach] [Email] [Out] 3 Re: Our partnership...,Email,"From: ""Ridge Driscoll"" <> To: ""Brad Migneron"" ..."


## 2️. Export Sample for Manual Labeling

The sampled subset is exported to a CSV file.
A human annotator will add a `human_label` column for evaluation purposes.


In [ ]:
df_eval.to_csv("human_annotation_sample.csv", index=False)

## 3. Generate Model Predictions for the Sample

Once the human-labeled data is ready, we run our LLM classifier on the same subset.
This ensures we can directly compare the **model predictions** (`trueai_engaged`) against the **ground truth** (`human_label`).


In [ ]:
# Reload human-labeled data
df_eval = pd.read_csv("human_annotation_labeled.csv", nrows=30)

# Run LLM classification
df_eval_predictions = run_full_process(df_eval, batch_size=20)

df_eval_predictions.head()


Processing rows 0 to 20
Processing rows 20 to 40


,hs_object_id,trueai_engaged
0,0,NOT ENGAGED
1,1,NOT ENGAGED
2,2,NOT ENGAGED
3,3,NOT ENGAGED
4,4,NOT ENGAGED


# Quantitative Evaluation (Metrics)

Now we compare: **human_label**  vs  **trueai_engaged**

## 4. Accuracy

We compute **overall accuracy** as the percentage of records where
the LLM prediction matches the human label.

This provides a high-level measure of model performance.


In [ ]:
# Merge df_eval (which contains human_label) with df_eval_predictions
# to bring the human_label into df_eval_predictions for comparison.
df_eval_predictions = df_eval_predictions.merge(
    df_eval[['hs_object_id', 'human_label']],
    on="hs_object_id",
    how="left"
)

accuracy = (
    df_eval_predictions["human_label"]
    == df_eval_predictions["trueai_engaged"]
).mean()

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 73.33%


## 5. Confusion Matrix

The confusion matrix shows the distribution of **true vs predicted labels**.
It helps identify:
- False ENGAGED → predicted engagement that didn’t occur
- False NOT ENGAGED → missed engagement signals
- ND errors → ambiguity handling

This is critical for understanding misclassification patterns.


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

labels = ["ENGAGED", "NOT ENGAGED", "ND"]

# Fill any potential NaN values in the label columns with 'ND'
df_eval_predictions["human_label"] = df_eval_predictions["human_label"].fillna("ND").astype(str)
df_eval_predictions["trueai_engaged"] = df_eval_predictions["trueai_engaged"].fillna("ND").astype(str)

cm = confusion_matrix(
    df_eval_predictions["human_label"],
    df_eval_predictions["trueai_engaged"],
    labels=labels
)

confusion_df = pd.DataFrame(cm, index=labels, columns=labels)
confusion_df

,ENGAGED,NOT ENGAGED,ND
ENGAGED,5,2,0
NOT ENGAGED,0,17,0
ND,0,5,1


## 6. Per-Class Precision, Recall, and F1-Score

We compute standard classification metrics per class:
- **Precision** → proportion of predicted positives that are correct
- **Recall** → proportion of actual positives correctly identified
- **F1-score** → harmonic mean of precision and recall

Focus is especially on ENGAGED recall since missing true engagement
can impact business decisions.


In [ ]:
from sklearn.metrics import classification_report

print(
    classification_report(
        df_eval_predictions["human_label"],
        df_eval_predictions["trueai_engaged"],
        labels=labels
    )
)


              precision    recall  f1-score   support

     ENGAGED       1.00      0.71      0.83         7
 NOT ENGAGED       0.71      1.00      0.83        17
          ND       1.00      0.17      0.29         6

    accuracy                           0.77        30
   macro avg       0.90      0.63      0.65        30
weighted avg       0.83      0.77      0.72        30



Interpretation:

ENGAGED recall → did we miss real conversations?

NOT ENGAGED precision → did we incorrectly flag outreach as success?

ND precision → is ambiguity handled responsibly?

## 7. Error Analysis

We inspect records where the model prediction **differs from the human label**.
This allows us to:
- Understand common failure modes
- Improve prompts or rules
- Document LLM limitations

We can filter for critical errors, e.g., false ENGAGED, which are
most impactful for sales reporting.


In [ ]:
# Merge descriptive columns from df_eval into df_eval_predictions for error analysis
df_eval_predictions = df_eval_predictions.merge(
    df_eval[['hs_object_id', 'crm_subject', 'crm_subtype', 'crm_description']],
    on='hs_object_id',
    how='left'
)

errors = df_eval_predictions[
    df_eval_predictions["human_label"]
    != df_eval_predictions["trueai_engaged"]
]

errors.head(10)


,hs_object_id,trueai_engaged,human_label,crm_subject,crm_subtype,crm_description
1,1,NOT ENGAGED,ND,1 - Medtronic - CA - **,Task,NaN
3,3,NOT ENGAGED,ND,4 - Medtronic - NH,Task,NaN
4,4,NOT ENGAGED,ND,convert,Task,NaN
5,5,NOT ENGAGED,ND,Email 1 - AMG Affiliate Messaging,Task,NaN
13,13,NOT ENGAGED,ND,[Outreach] [Email] [In] Scheduled - ReRoute Co...,Email,From: RingLead Customer Success\nTo: kharrell@...
19,19,NOT ENGAGED,ENGAGED,[Outreach] [Email] [Out] 7 marketing collateral,Email,"From: ""Ryan Du"" <rdu@verifictus.com>\nTo: ""Ang..."
26,26,NOT ENGAGED,ENGAGED,Re: verifictus Renewal Q-31033,Task,"Hi Paul,\n\n \n\nJust wanted to send a nudge ..."


In [ ]:
false_engaged = errors[
    (errors["trueai_engaged"] == "ENGAGED") &
    (errors["human_label"] != "ENGAGED")
]

false_engaged[
    ['crm_subject', 'crm_subtype', 'crm_description']
].head(5)


,crm_subject,crm_subtype,crm_description


## 8. Comparison with Rule-Based Labels

We can also compare LLM predictions against our **pre-labeling heuristic rules**.
This gives insight into:
- Cases where the LLM improves on simple rules
- Consistency between automated heuristics and AI predictions


In [ ]:
agreement = (
    df_final["classification"]
    == df_final["trueai_engaged"]
).mean()

print(f"Agreement with heuristics: {agreement:.2%}")

Agreement with heuristics: 34.00%


## 9. Summary of Evaluation

- Human-labeled evaluation set: ~200 activities
- Metrics computed: Accuracy, Precision, Recall, F1, Confusion Matrix
- Error analysis performed for misclassifications
- ND class effectively captures ambiguity
- Results guide **prompt improvements** and **model fine-tuning**
